# NASA Space Apps 2020: COVID-19 Challenge

## Factor Analysis.

<img src="../reports/figures/poverty_dimensions.jpeg" width=40%/>

Importante considerations:
1. There are no outliers in data.
2. Sample size should be greater than the factor.
3. There should not be perfect multicollinearity.
4. There should not be homoscedasticity between the variables.

In [6]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from glob import glob
%matplotlib inline

dados_tratados = glob("../data/external/*.csv")

### Listagem de dados tratados

In [ ]:
# Google Colab POG
from google.colab import drive
drive.mount('/gdrive')
%cd "/gdrive/My Drive/Hackatons/NASA Space Apps 2020"
dados_tratados = glob("Data/Tratados/*.csv")

In [7]:
fnames = list(map(os.path.basename, dados_tratados))
wlength = np.max(list(map(len,fnames)))
dados_tratados = dict(zip(fnames, dados_tratados))
for k,v in dados_tratados.items():
  print((k+":").rjust(wlength), v)

                    AguaTratada.csv: ../data/external\AguaTratada.csv
                   BasicHygiene.csv: ../data/external\BasicHygiene.csv
                      Desastres.csv: ../data/external\Desastres.csv
                   HospitalBeds.csv: ../data/external\HospitalBeds.csv
                            IDH.csv: ../data/external\IDH.csv
                   Immunization.csv: ../data/external\Immunization.csv
                    Local_x_USD.csv: ../data/external\Local_x_USD.csv
                        Medicos.csv: ../data/external\Medicos.csv
                 OpenDefecation.csv: ../data/external\OpenDefecation.csv
                      Populacao.csv: ../data/external\Populacao.csv
REAL_GDP_PER_CAPITA_CONSTANT_US$.csv: ../data/external\REAL_GDP_PER_CAPITA_CONSTANT_US$.csv
                   RedeDeEsgoto.csv: ../data/external\RedeDeEsgoto.csv
                  SalarioMinimo.csv: ../data/external\SalarioMinimo.csv


In [8]:
df_populacao = pd.read_csv(dados_tratados["Populacao.csv"], index_col=0)
df_populacao.sort_values(["Ano", "País"], ignore_index=True, inplace=True)
print(df_populacao["Ano"].unique())
df_populacao.head()

[2010]


,País,População,Ano
0,ARGENTINA,4.011689e+07,2010
1,BOLIVIA,9.684295e+06,2010
2,BRASIL,1.907474e+08,2010
3,CHILE,1.709418e+07,2010
4,COLOMBIA,4.836490e+07,2010


In [9]:
df_hospital_beds = pd.read_csv(dados_tratados["HospitalBeds.csv"], index_col = 0)
df_hospital_beds.sort_values(["year", "country"], ignore_index=True, inplace=True)
print(np.sort(df_hospital_beds["year"].unique()))
df_hospital_beds.head()

[2013 2014 2015]


,country,year,hospital_beds
0,Chile,2013,2.2
1,Ecuador,2013,1.5
2,Argentina,2014,5.0
3,Bolivia,2014,1.1
4,Brazil,2014,2.2


In [10]:
df_medicos = pd.read_csv(dados_tratados["Medicos.csv"], index_col = 0)
df_medicos.sort_values(["Period", "Location"], ignore_index=True, inplace=True)
print(df_medicos["Location"].unique())
print(df_medicos["Period"].unique())
df_medicos.head()

['Venezuela (Bolivarian Republic of)' 'Bolivia (Plurinational State of)'
 'Ecuador' 'El Salvador' 'Peru' 'Argentina' 'Mexico' 'Brazil' 'Chile'
 'Colombia']
[2001 2016 2017 2018]


,Location,Period,Indicator,First Tooltip
0,Venezuela (Bolivarian Republic of),2001,"Medical doctors (per 10,000)",19.48
1,Bolivia (Plurinational State of),2016,"Medical doctors (per 10,000)",15.90
2,Ecuador,2016,"Medical doctors (per 10,000)",20.37
3,El Salvador,2016,"Medical doctors (per 10,000)",15.66
4,Peru,2016,"Medical doctors (per 10,000)",13.05
